# HK PROTESTS: Visualising Chinese State Troll Tweets, Part 3 (Chinese Text)

In Parts 3 and 4, I'll re-trace the steps in the previous two sections but for the Chinese tweets this time.

I had problems plotting the frequency token distribution and tree map charts for Chinese characters, and decided to use Google Sheets instead to plot the key terms to speed up completion of this section. I'll update this notebook if I fix the font issue at some point. 

In [1]:
import jieba, jieba.analyse
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

mpl.rcParams["figure.dpi"] = 300
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [2]:
pd.set_option('display.max_columns', 40)

# 1. "DE-LAYERING" THE CHINESE TROLL TWEETS

Steps below are broadly similar to Part 1, with some modifications for Chinese text, which has no word spacing, unlike English. I used the [jieba](https://github.com/fxsjy/jieba) word segmentation module for this projection. There are other options for Chinese NLP tasks, including modules from Stanford, Fudan etc. I would welcome feedback on whether adopting different Chinese NLP models/modules would significantly affect the outcomes here.

## 1.1 DATA PRE-PROCESSING
Download the original CSV files from [Twitter](https://blog.twitter.com/en_us/topics/company/2019/information_operations_directed_at_Hong_Kong.html). 

In [3]:
# Reminder that the raw1/2 CSV files are NOT in this repo. Download directly from Twitter; link above
raw1 = pd.read_csv('../data/china_082019_1_tweets_csv_hashed.csv')
raw2 = pd.read_csv('../data/china_082019_2_tweets_csv_hashed.csv')
raw = pd.concat([raw1, raw2])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15,19,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Dropping unnecessary columns
raw = raw.drop(
    columns=[
        "user_profile_url",
        "tweet_client_name",
        "in_reply_to_tweetid",
        "in_reply_to_userid",
        "quoted_tweet_tweetid",
        "is_retweet",
        "retweet_userid",
        "retweet_tweetid",
        "latitude",
        "longitude",
        "quote_count",
        "reply_count",
        "like_count",
        "retweet_count",
        "urls",
        "user_mentions",
        "poll_choices",
        "hashtags",
    ]
)

In [5]:
# Converting timings to HK time, and extracting year-month-day-hour cols
raw['tweet_time'] = pd.to_datetime(raw['tweet_time'])
raw['tweet_time'] = raw['tweet_time'].dt.tz_localize('GMT').dt.tz_convert('Hongkong')
raw['tweet_year'] = raw['tweet_time'].dt.year
raw['tweet_month'] = raw['tweet_time'].dt.month
raw['tweet_day'] = raw['tweet_time'].dt.day
raw['tweet_hour'] = raw['tweet_time'].dt.hour

raw['account_creation_date'] = pd.to_datetime(raw['account_creation_date'], yearfirst=True)
raw['year_of_account_creation'] = raw['account_creation_date'].dt.year
raw['month_of_account_creation'] = raw['account_creation_date'].dt.month
raw['day_of_account_creation'] = raw['account_creation_date'].dt.day

## 1.2 DATA FILTERING + CLEANING

In [6]:
# I'll focus only on tweets sent from 2017
raw = raw[(raw["tweet_year"] >= 2017)].copy()

In [7]:
# Filtering out tweets which mention fugitive Chinese billionaire Guo Wengui, 
# and other irrelevant characters like US-based dissidents Yang Jianli, Guo Baosheng etc
raw = raw[~raw["tweet_text"].str.contains("郭文贵")].copy()
raw = raw[~raw["tweet_text"].str.contains("文贵")].copy()
raw = raw[~raw["tweet_text"].str.contains("郭文")].copy()
raw = raw[~raw["tweet_text"].str.contains("杨建利")].copy()
raw = raw[~raw["tweet_text"].str.contains("郭宝胜")].copy()
raw = raw[~raw["tweet_text"].str.contains("宝胜")].copy()
raw = raw[~raw["tweet_text"].str.contains("老郭")].copy()
raw = raw[~raw["tweet_text"].str.contains("郭狗")].copy()
raw = raw[~raw["tweet_text"].str.contains("郭骗子")].copy()
raw = raw[~raw["tweet_text"].str.contains("余文生")].copy()
raw = raw[~raw["tweet_text"].str.contains("吴小晖")].copy()

In [8]:
# In this notebook, I'll focus only on Chinese tweets. 
# In earlier drafts, I found that troll accounts with English language settings were sending out Chinese tweets too,
# so provisions were made here to include those 
# Note the sub-categories for Twitter language settings for English and Chinese

raw_ch = raw[
    (raw["tweet_language"] == "zh")
    & (
        (raw["account_language"] == "en")
        | (raw["account_language"] == "en-gb")
        | (raw["account_language"] == "zh-cn")
        | (raw["account_language"] == "zh-CN")
        | (raw["account_language"] == "zh-tw")
    )
].copy()

In [9]:
raw_ch.shape 

(121962, 20)

In [10]:
# Splitting the dataset into two sub-sets, one for retweets and one for "original" tweets
raw_ch_no_rt = raw_ch[~raw_ch["tweet_text"].str.startswith("RT @")].copy()

raw_ch_rt = raw_ch[raw_ch["tweet_text"].str.startswith("RT @")].copy()

In [11]:
# Surprisingly, the Chinese language tweets subset is smaller than the English one
raw_ch_no_rt.shape, raw_ch_rt.shape

((106957, 20), (15005, 20))

In [12]:
# Define a simple function to clean the tweet_text column and weed out non-Chinese text
def clean_tweet_ch(text):
    text = text.strip(" ")
    text = text.strip(r"\n")
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"http\S+", "", text)
    filtered = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    text = filtered.sub(r'', text) # remove all non-Chinese characters
    return text

In [13]:
raw_ch_no_rt['clean_tweet_text'] = raw_ch_no_rt['tweet_text'].map(lambda tweet: clean_tweet_ch(tweet))

raw_ch_rt['clean_tweet_text'] = raw_ch_rt['tweet_text'].map(lambda tweet: clean_tweet_ch(tweet))

In [14]:
# Uncomment for a sample of the cleaned up "original" Chinese tweets
#raw_ch_no_rt['clean_tweet_text'].sample(5)

In [15]:
# Uncomment for a sample of the cleaned up Chinese retweets
# raw_ch_rt['clean_tweet_text'].sample(5)

# 2. DIGGING THROUGH LAYER#1 - SPAM TWEETS GALORE BUT A HINT OF CONTENT TARGETTING THE PROTEST MOVEMENT

Before plotting out the frequency distribution of key words, I settled on a brief list of Chinese stop words. They are not exhaustive by any means, and reflect words which I wanted to weed out following initial drafts.

In [16]:
jb_stopwords = [
    "的",
    "了",
    "和",
    "是",
    "就",
    "都",
    "而",
    "及",
    "與",
    "著",
    "或",
    "一個",
    "沒有",
    "我們",
    "你們",
    "妳們",
    "他們",
    "她們",
    "是否",
    "时间",
    "整点",
    "报时",
    "现在",
    "日电",
    "月",
    "日",
    "桂",
    "海",
    "在",
    "电"
]

In [17]:
text = raw_ch_no_rt["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))

Building prefix dict from /anaconda3/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/6z/wrz4dxdx65585cc04rbtr1xh0000gn/T/jieba.cache
Loading model cost 1.025320053100586 seconds.
Prefix dict has been built succesfully.


In [18]:
# CountVect for subset with "original" tweets
# I raised the ngram_range to (2,4) and min_df to 10 after several rounds of trial and error. 
# Dial both down if you wish to see the count for single words

vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)
docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list1 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list1.sort_values(ascending = False).head(50)

一个 人           1597
谷 歌             918
微 信             704
不 知道            592
虎 扑             578
安 卓             570
之 家             562
扑 讯             528
虎 扑 讯           526
外 媒             522
让 你             516
每个 人            512
喜剧 人            503
更 多             497
世界 上            470
将 于             467
一生 平安           462
好人 一生 平安        462
好人 一生           462
平安 好人           460
你 会             458
也 会             447
让 人             443
如果 你            441
会 有             439
你 若             431
欢乐 喜剧 人         424
欢乐 喜剧           424
太 多             412
骁 龙             398
微 博             388
晒 日光浴           386
日光浴 舒服          385
晒 日光浴 舒服        385
好人 一生 平安 好人     385
平安 好人 一生        385
一生 平安 好人 一生     385
一生 平安 好人        385
平安 好人 一生 平安     385
也 不             374
给 你             363
当 你             358
魅 族             357
我 不             354
说 我             352
你 不             348
晒 日光浴 舒服 晒      340
舒服 晒 日光浴        340
舒服 晒            340
日光浴 舒服 晒        340


<img src="tweets_ch01.jpg">

In [19]:
# raw_ch_no_rt[raw_ch_no_rt["clean_tweet_text"].str.contains('一个人')].values
# raw_ch_no_rt[raw_ch_no_rt["clean_tweet_text"].str.contains('谷歌')].values
# raw_ch_no_rt[raw_ch_no_rt["clean_tweet_text"].str.contains('日光浴')].values
# raw_ch_no_rt[raw_ch_no_rt["clean_tweet_text"].str.contains('平安')].values

# QUICK TAKE: 
Like the English tweets, the lightly filtered Chinese tweets subset is extremely noisy as the chart above shows. The most frequently used words in "original" tweets did not throw up phrases targetted at the HK protest movement. The tweets are instead replete with spam content involving crime stories, US-China relations, and sunbathing, of all things.....Uncomment the cells above for a flavour of the spam content in the tweets.

The retweets subset below, however, threw up early hints of the content aimed directly at the protest movement, with words like "反送中" (anti-extradition bill), "香港警察" (Hong Kong police) and "遊行" (street rallies/protests) appearing in the top 50 list. 

In [20]:
# CountVect for subset with only retweets
text = raw_ch_rt["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))
    
vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)
docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list2 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list2.sort_values(ascending = False).head(50)    

严重 暴力犯罪          246
严重 暴力犯罪 案件       206
暴力犯罪 案件          206
依法 侦办            204
公安机关 依法          198
公安机关 依法 侦办       196
中国 人             178
一起 严重 暴力犯罪       165
一起 严重            165
一起 严重 暴力犯罪 案件    165
中 美              161
送 中              157
侦办 一起            151
侦办 一起 严重         151
侦办 一起 严重 暴力犯罪    151
中 國              149
依法 侦办 一起         148
依法 侦办 一起 严重      148
政治 庇护            145
公安机关 依法 侦办 一起    140
香港 警察            140
反 送 中            140
反 送              140
川 普              136
反 對              131
破 壞              121
遊 行              120
立法 會             120
不 知道             119
采取 刑事            118
瘟 鬼              117
刑事 强制措施          114
采取 刑事 强制措施       114
菜谱 作者            113
社 會              110
撐 警              109
犯罪 嫌疑人           108
逃犯 條             106
條 例              106
名 犯罪 嫌疑人         105
名 犯罪             105
逃犯 條 例           104
阿 贵              102
推 友              101
无 语              100
你 还               99
还 能               98
被 依法         

In [21]:
# Uncomment the lines below for a flavour of the spam content in the retweet subset
#raw_ch_rt[raw_ch_rt["clean_tweet_text"].str.contains('中美')].values
#raw_ch_rt[raw_ch_rt["tweet_text"].str.contains('中国')].values

<img src="retweets_ch01.jpg">

# 3. DIGGING THROUGH LAYER#2 - TWEETS ON CHINESE DISSIDENTS/FUGITIVES, COMEDIES, AND US-CHINA RELATIONS

Like in Part 1, I opted to filter directly for terms which would be more relevant to the analysis, given the amount of noise in the dataset.

Following earlier trials, which I won't include here, I opted for the terms 香港(Hong Kong), 政府(government), 中国(China), 顏色革命(Colour Revolution), 外國勢力(Foreign Forces), and 警察 (police).

## 3.1 SIFTING THROUGH "ORIGINAL" CHINESE TROLL TWEETS 

In [22]:
ch_no_rt1 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("香港")].copy()
ch_no_rt2 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("政府")].copy()
ch_no_rt3 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("中国")].copy()
ch_no_rt4 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("顏色革命")].copy()
ch_no_rt5 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("外國勢力")].copy()
ch_no_rt6 = raw_ch_no_rt[raw_ch_no_rt['tweet_text'].str.contains("警察")].copy()

hk_ch_no_rt = pd.concat([ch_no_rt1, ch_no_rt2, ch_no_rt3, ch_no_rt4, ch_no_rt5, ch_no_rt6])

In [23]:
# There are 412 unique users in this subset consisting of "original" tweets
hk_ch_no_rt['user_screen_name'].nunique()

412

In [24]:
# We see some familiar troll accounts pop up again, particularly HKpoliticalnew and ctcc507
# Both accounts were active in the English set in Part 1 as well
hk_ch_no_rt['user_screen_name'].value_counts().head(10)

ueV2dN3HOL24gOeDeNDZzDn0LU+68V9kby+jDR2pm4=     1085
ingaibragimova1                                  834
HKpoliticalnew                                   648
charikamci                                       551
mauricerowleyx                                   469
zGKXuesfHo+nPb6rrSG61fpwFuGLKslqTK6weUoKWTI=     392
KondratevFortu                                   160
ctcc507                                          159
bagaudinzhigj                                    126
gdvcgsfsg                                        124
Name: user_screen_name, dtype: int64

In [25]:
# This troll account was very active in the English subset as well
hk_ch_no_rt[hk_ch_no_rt['user_screen_name']=='ctcc507']['tweet_text'].head().values

array(['@SuiBianXiaoGe 或许一觉醒来，或许明天，或许不久的将来，香港人就会明白不论是运行《逃犯条例》还是暂停修订《逃犯条例》都是中国对香港人民的爱和包容，都是为了维护香港治安稳定和香港人“民主”的良苦用心。而西方反华势力试图在香港推动的颜色革命最终会被中国人民齐心协力扼杀在摇篮中。',
       '港版微信支付升级！“无现金”生活还远吗？ #微信 支付近日宣布，在香港推出一系列升级支付功能，标志着香港在“无现金”支付的路上又迈进了一步。 香港消费者通过微信扫描二维码可完成线上线下、不同消费场景的支付，而香港的士、报刊亭、超… https://t.co/mblIgvjTIK https://t.co/JC4uxCwltT',
       '11月2日下午消息，双11前夕，支付宝在海外在线支付陆续落地。许多国家和地区的支付机构提前几个月，就开始联合支付宝升级自己的在线支付网络，菲律宾和香港的本地钱包今年还将首次参与“双11”。',
       '認識傳染性極強的麻疹：紐約、泰國、香港各地紛紛爆發，有哪些症狀該如何預防？ https://t.co/Tg9UhwAVa8',
       '10月30日， #香港科技大学 宣布，该校机器人研究所的师生研发了香港首部拥有多项创新功能的 #无人车。 此外，他们还研发了一个特别设计的控制台，能统一控制无人车的动态功能，包括电线驱动的转向、加速及制动功能。 https://t.co/E0R2ryskkl https://t.co/WzK4rcR2Ee'],
      dtype=object)

In [26]:
# The key words filter don't always accurately filter for content targetting the protest movement
# This user's tweets seemed mostly irrelevant
hk_ch_no_rt[hk_ch_no_rt['user_screen_name']=='ingaibragimova1']['tweet_text'].head().values

array(['香港将修例禁止电子烟：违者最高罚5万港元及监禁半年 https://t.co/NlL2VYV28X',
       '特斯拉在香港建立充电站：有50个充电位 为亚洲最大 https://t.co/fZCvguZaSy',
       '中国联通宣布与香港、澳门电讯合作：精品网延迟1毫秒 https://t.co/NKahtMVhUM',
       '消息称香港将向6家公司发放数字银行牌照 腾讯小米等在内 https://t.co/AlxLAnTYM0',
       '香港中文大学开设首个人工智能学位课程 https://t.co/xeloTj0bGX'], dtype=object)

In [27]:
# This troll account was very active in the English subset as well
hk_ch_no_rt[hk_ch_no_rt['user_screen_name']=='HKpoliticalnew']['tweet_text'].head().values

array(['千名黑衣人圍堵立會，當中不乏重裝上陣的人，警方一定要嚴防，大家小心！👍💪\n\n#香港 #大專學界 #升級行動 #包圍立會 #重裝上陣 #警察 \n原圖：星島日報 https://t.co/DIoiFWWkBo',
       '立會經暴亂一役，幾近淪為廢墟。\n\n據《星島日報》報道，昨日，警方組織罪案及三合會調查科到立會蒐證調查，已掌握十名暴徒資料，他們事涉暴力衝擊包圍等事，並於日內拉人。本報於早上亦曾報道，執法者或會拘捕起警底作欺凌者。\n\n#香港 #立法會大樓 #星島日報 #暴徒 #日內拉人 https://t.co/m0KCaYLAkx',
       '作為教育工作者理應「傳道、授業、解惑」，對於修訂《逃犯條例》這個關係香港法治的重大議題，更應該對學生負起「講清楚、講明白」的責任。\nhttps://t.co/XgbK2aSG6s港人博評/45389/反修例誤導年輕人走上暴力衝擊邪路?fbclid=IwAR0tp45O3W7PdltdwfWaFlAvkweUfMEiImn8nyrF61vQVvLQwu3p6PoPZk8#selected',
       '#西班牙 嘅警察對付巴斯克分離份子。棍棍打下去，絕唔手軟，完全合法，絕無手尾。\n對比之下，系唔系覺得 #香港警察 確實已經很溫柔了？ https://t.co/rhHIXiPUn8',
       '#香港 反對派一再散布失實資訊抹黑修例、誤導市民，更不斷造謠煽動情緒，企圖以網民壓力杯葛支持修例甚至只是沉默的商戶。\n政府決定停止修例工作後，反對派又繼續播謠，稱換領智能身份證會「失去選民資格」，企圖撕裂市民對社會不同界別、團體、組織的信任。 https://t.co/9B3xCI9MWv'],
      dtype=object)

In [28]:
# Sticking to the same min_df and ngram_range values

text = hk_ch_no_rt["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))
    
vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)

docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list3 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list3.sort_values(ascending = False).head(50)  

條 例       229
特 區       218
中国 人      212
港 獨       197
反 對       194
立法 會      189
香港 警察     183
美 國       180
區 政府      177
特 區 政府    172
逃犯 條 例    170
逃犯 條      170
中国 驻      164
社 會       162
中 美       158
对 中国      147
反 對 派     140
對 派       140
中 國       138
民族 黨      135
林 鄭       129
香港 特      125
總 部       120
修 例       119
行 動       117
香港 特 區    117
专项 行动     113
勢 力       112
學 生       112
發 展       110
會 議       109
与 中国      108
香港 人      108
中国 法律     107
記 者       106
外 國       106
修 訂       105
逃犯 条例     103
議 員       100
撐 警        99
中国 游客      98
國 家        97
大 學        97
两 国        94
鄭 娥        93
林 鄭 娥      93
香港 民族      93
一 個        91
內 地        90
外 媒        90
dtype: int64

<img src="tweets_ch02.jpg">

In [29]:
# hk_ch_no_rt[hk_ch_no_rt["tweet_text"].str.contains('反對派')].values
# hk_ch_no_rt[hk_ch_no_rt["tweet_text"].str.contains('香港警察')].values
# hk_ch_no_rt[hk_ch_no_rt["tweet_text"].str.contains('港獨')].values

# QUICK TAKE: 

The most active trolls, as we can see here and in Part 1, are posting content in English and Chinese, the two primary languages at play here. HKpoliticalnew and ctcc507, for instances, are tweeting in both languages.

The direct filtering also gives us a better sense of the messages that the trolls were trying to push. A sample:

- @charikamci, tweeting at 2018-08-19 00:26:00+08:00: '香港回歸至今反對派不時興風作浪年爆發日違法佔中反對派倒打一耙將責任推到特區政府和中央身上大公報獲得的最新機密文件顯示佔中其實是亂港派與境外反華勢力精心策動的一場顏色革命旨在利用年輕人妄圖一夜變天']

- @Resilceale; tweeting at 2019-07-09 12:17:00+08:00: 以「反修例」為借口，以學生為棋子，以打砸為手段，以分化為目的，香港反對派不惜重金招募，幕後指使學生 「沖擊立法會」，讓「東方明珠」嘅天空布滿陰霾，港民應擦亮慧眼！"

- @qujianming1; tweeting at 2019-07-02 10:14:00+08:00: '香港七一游行民阵月日反对派宣扬暴力冲击立法会的举动是对香港百年法治精神的践踏将和平游行演变成颜色革命令人愤慨此时刻香港市民应该保持克制冷静用理性和平的方式表达意见携手香港警察共同维护社会稳定社会各界更应冷静理性共同促进香港法治进步'

- @maksmkas6g; tweeting at 2019-07-04 17:11:00+08:00: '@bindarsou 打掉这帮暴徒的嚣张，香港警察加油'

- @Resilceale, tweeting at 2019-07-09 12:15:00+08:00: '香港嘅「港獨」分子，為了達到不可告人之目的，煽動不明真相嘅青年人參與游行，甚至進行所謂嘅打砸搶違法行為向政府施壓。仲系醒醒吧，少干點蠢事，多長點心眼！世界上冇邊個國家允許呢種猖狂行徑盛行嘅，唔系唔打，肯定要打掉嘅！而且要重打！

Uncomment the lines above to see a fuller set of the tweets. The content in itself is not surprising - essentially backing the Hong Kong police and blaming secessionist groups and "hostile foreign forces" for trying to forment a colour revolution.

A detailed content analysis, however, is outside the scope of this notebook.

## 3.2 SIFTING THROUGH CHINESE TROLL RETWEETS 

In [30]:
ch_rt1 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("香港")].copy()
ch_rt2 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("政府")].copy()
ch_rt3 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("中国")].copy()
ch_rt4 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("顏色革命")].copy()
ch_rt5 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("外國勢力")].copy()
ch_rt6 = raw_ch_rt[raw_ch_rt['tweet_text'].str.contains("警察")].copy()

hk_ch_rt = pd.concat([ch_rt1, ch_rt2, ch_rt3, ch_rt4, ch_rt5, ch_rt6])

In [31]:
hk_ch_rt['user_screen_name'].nunique()

254

In [32]:
hk_ch_rt['user_screen_name'].value_counts().head(10)

FWE41OnopBB5h3unPH3s3XBA3t3zEuROlhnEue2H8cE=    369
benjaminkudla39                                 110
HKpoliticalnew                                   62
GirgRKG36vs7eBx81goMAn6AlVfUp0RjKLwCCdd7aU=      60
ardansuweb                                       60
randxr89c                                        44
w9pbfQRUXBcO810z7Q9I5TbWnGdbZaBB3Gvh6KxT6Y=      39
fMPA8G2z8yjkrEMdZkNSnRraECis1zX6tv2N7NcF7aY=     34
2l1eDka0eiClBUYoDXlwYaKcUaeelnz44aDM9OJRM=       31
Melissa64269411                                  28
Name: user_screen_name, dtype: int64

In [33]:
# There's still noise in these filtered tweets, evidenced by the tweets on a fashion festival and art auction
hk_ch_rt[hk_ch_rt['user_screen_name']=='ardansuweb']['tweet_text'].head().values

array(['RT @simsoer: 曾經亞洲最安全城市嘅締造者-香港警察，為了最小嘅傷害保持了最大嘅克制。 嚴正聲明——強力支持香港警察嘅執法行為，強烈譴責香港反修例暴徒暴力嘅行為，愿香港重歸穩定，齊心建設粵港澳灣區 #HK #HongKongProtest \nhttps://t.co/…',
       'RT @CNS1952: 佳士得香港2019年春季拍卖会将于24日在香港会展中心开槌。为期一周的春拍将呈献中国书画、中国瓷器及艺术品、亚洲二十世纪及当代艺术、珠宝名表等多个专场，拍品横跨古代至当代的东西方艺术。 https://t.co/BtQYX2Egyr',
       'RT @CNS1952: 7月8日，由香港贸易发展局主办的第26届香港时装节春夏系列假湾仔香港会议展览中心举行。一连四日的展览以Oriental Fever为布展主题，吸引来自12个国家及地区约一千家参展商参与。来看看开幕时装巡礼表演吧！ https://t.co/Et045p…',
       'RT @CNS1952: 5月22日，中国一支民间女子登山队成功登顶珠穆朗玛峰。分别来自新疆、香港、河南的马丽娅姆、曾燕红、孙宁相继登顶。从4月8日出发至今，她们历经40多天到达世界之巅。目前，女子登山队的全部成员正在安全下撤途中。#中国故事 https://t.co/q9JT…',
       'RT @CNS1952: 香港特别行政区一些极端激进分子1日以极为暴力的方式冲击立法会大楼，肆意损坏立法会设施，海外专家学者对此纷纷发声谴责，认为这是践踏法治、危害社会秩序的严重违法行为，并呼吁外国势力停止插手香港事务和中国内政。https://t.co/NDVO6CxZZl…'],
      dtype=object)

In [34]:
hk_ch_rt[hk_ch_rt['user_screen_name']=='randxr89c']['tweet_text'].head().values

array(['RT @aptib08e7: #香港 #hongkong 睇到暴力行為和活動，首先我認為作為一個香港人，就應該從自己做起，從一個維護香港社會大局穩定嘅角度出發，自覺抵觸呢種暴力嘅活動，並大聲嘅呼吁社會各界盡快冷靜落嚟，唔好參與諸如此類嘅活動，香港需要嘅系和平、發展、繁榮，而唔系…',
       'RT @simsoer: 曾經亞洲最安全城市嘅締造者-香港警察，為了最小嘅傷害保持了最大嘅克制。 嚴正聲明——強力支持香港警察嘅執法行為，強烈譴責香港反修例暴徒暴力嘅行為，愿香港重歸穩定，齊心建設粵港澳灣區 #HK #HongKongProtest \nhttps://t.co/…',
       'RT @anafedinzp: #HongKong #HK 眾所周知，香港警隊作為維護香港安定嘅守護者往往承擔著比其他職業更多嘅社會責任和壓力。面對近期呢一系列衝突事件，香港警方面對已成騷亂嘅暴力行徑被迫採取咗必要嘅處置手段，既保持咗剋制，態度又堅決。我哋認為，香港警方表現出嘅…',
       'RT @CNS1952: 原定28日进行拍卖的清代画家任伯年画作《澹黄杨柳带栖鸭》于26日预展时被一名小童撕毁。佳士得香港方面称已通知委托方，并撤拍该作品。被毁的画作为创作于1889年的《花鸟四屏》中的一幅。整组作品估价为150万至250万港元（约合19至31万美元）。http…',
       'RT @CNS1952: 2019香港玩具节16日在香港会展中心继续举行。本届玩具节吸引众多玩具迷蜂拥入场，除了儿童以外，亦有众多成年玩具迷前来参观、选购潮流玩具和怀旧玩具。 https://t.co/qnUBRQPX58'],
      dtype=object)

In [35]:
text = hk_ch_rt["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))
    
vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)

docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list4 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list4.sort_values(ascending = False).head(50) 

香港 警察       285
送 中         207
撐 警         205
反 對         199
中国 人        189
反 送         181
反 送 中       181
行 動         159
社 會         156
逃犯 條        137
條 例         137
逃犯 條 例      135
破 壞         131
行 為         127
遊 行         126
我 哋         126
香港 撐        117
對 派         117
反 對 派       117
立法 會        115
中 香港        115
送 中 香港      115
執 法         112
修 例         110
暴 動         107
警察 嘅        106
學 生         102
嚴 正         101
穩 定          96
警 隊          95
譴 責          92
反 送 中 香港     89
中 美          86
勢 力          83
警 行          83
警 行 動        83
撐 警 行 動      83
撐 警 行        83
反修 例         82
撐 警 隊        82
香港 撐 警 行     81
你 我          81
香港 撐 警       81
隊 你 我 同行     80
撐 警 隊 你      80
警 隊 你        80
警 隊 你 我      80
隊 你          80
你 我 同行       80
我 同行         80
dtype: int64

<img src="retweets_ch02.jpg">

In [36]:
# hk_ch_rt[hk_ch_rt["tweet_text"].str.contains('反對派')].values
# hk_ch_rt[hk_ch_rt["tweet_text"].str.contains('香港警察')].values
# hk_ch_rt[hk_ch_rt["tweet_text"].str.contains('港獨')].values

The retweets push a similar narrative - praising the police for taking action and condemning the violent protests. Uncomment the cell above for a fuller look. A sample: 

'RT @bindarsou: 香港警察真正嘅正義使者反對派污蔑攻击香港警察嘅目的係為咗擾亂香港嘅社會穩定相信廣大具有正義感嘅市民同學生定能睇清事實不再被反對派制造嘅假象迷惑譴責利用學生進行暴力嘅邋遢行為香港警察撐港警反暴力遊行'

'RT @simsoer: 曾經亞洲最安全城市嘅締造者香港警察為了最小嘅傷害保持了最大嘅克制嚴正聲明強力支持香港警察嘅執法行為強烈譴責香港反修例暴徒暴力嘅行為愿香港重歸穩定齊心建設粵港澳灣區'

'RT @GolloglyAlysha: #香港 #逃犯條例 看看美國是怎麼培養港獨勢力的這些所謂的青年精英無非是美國人的棋子禍港亂港的真兇'

Support for the HK police (with the phrase/hashtag "撐警", which has come under intense criticisms for its handling of the protests, appear to be more prominent in the rewteets subset.

# 4. DIGGING THROUGH LAYER#3 - THE TOP TROLLS

Like in Part 1, I'll sift through the accounts most active in tweeting or retweeting content targetting the HK protest movement. The 10 accounts I picked below are not exhaustive by any means.

## 4.1 SIFTING THROUGH TOP TROLL RETWEETS IN CHINESE

In [37]:
trolls_ch = [
    "HKpoliticalnew",
    "charikamci",
    "ctcc507",
    "KondratevFortu",
    "jdhdnchsdh",
    "shaunta58sh",
    "Resilceale",
    "qujianming1",
    "vezerullasav158",
    "ardansuweb"
]

In [38]:
# Starting with "original" tweets, in Chinese, by the top trolls
top_ch_trolls = hk_ch_no_rt[hk_ch_no_rt['user_screen_name'].isin(trolls_ch)]
top_ch_trolls.shape

(1817, 21)

In [39]:
text = top_ch_trolls["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))
    
vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)
docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list5 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list5.sort_values(ascending = False).head(50)  

特 區          191
港 獨          189
條 例          189
立法 會         181
美 國          171
反 對          160
區 政府         155
特 區 政府       152
逃犯 條 例       140
逃犯 條         140
民族 黨         133
林 鄭          129
香港 特         114
對 派          113
反 對 派        113
行 動          112
社 會          111
總 部          111
香港 特 區       109
學 生          107
外 國          106
記 者          106
會 議          104
勢 力          102
中 國          102
撐 警           97
發 展           97
修 訂           95
議 員           95
鄭 娥           93
林 鄭 娥         93
香港 民族         91
大 學           91
陳 浩           89
內 地           89
國 家           86
佔 中           86
香港 民族 黨       86
修 例           83
陳 浩 天         81
浩 天           81
色 革命          80
長 官           80
行政 長 官        80
行政 長          80
顏 色           76
顏 色 革命        75
警 總           74
活 動           74
香港 特 區 政府     72
dtype: int64

<img src="toptrolls_tweets.jpg">

In [40]:
# top_ch_trolls[top_ch_trolls["tweet_text"].str.contains('特區')].values
# top_ch_trolls[top_ch_trolls["tweet_text"].str.contains('港獨')].values
# top_ch_trolls[top_ch_trolls["tweet_text"].str.contains('美國')].values
# top_ch_trolls[top_ch_trolls["tweet_text"].str.contains('顏色革命')].values

## QUICK TAKE:
The top trolls are far more aggressive in pushing the conspiracy theories about the US and other "hostile foreign forces" being behind the HK protests. 

@HKpoliticalnew tweeted on July 1 2019, for instance, that the staff at the UK Consulate in HK, some whom it accused of being "CIA" agents, were behind efforts to destablise Hong Kong. On June 13, the same account also sent out a tweet alleging that foreign intelligence agents were working undercover as journalists in order to disrupt the work of the police.

Another user, @KondratevFortu alleged that Apple Daily founder Jimmy Lai had flown to the US to conspire with the US to "interfere" with Hong Kong's affairs. The tweet was sent on the same day where the [Chinese foreign ministry slammed Mr Lai and top US officials](https://www.scmp.com/news/hong-kong/politics/article/3017868/beijing-foreign-office-slams-hong-kong-tycoon-jimmy-lai-and) for interfering in HK and China's "internal affairs".

A sample of the tweets below. Uncomment the cell above to see the tweets in full:

- @HKpoliticalnew; tweeting at 2019-06-18 16:38:00+08:00: 美國資助「港獨」廢青洗腦\n#諜影重重 #顏色革命 #香港 https://t.co/qZvsZQpwoa'

- @HKpoliticalnew; tweeting at 2019-07-01 05:12:00+08:00: '美國香港領使館千人員工，不小是CIA特工連同香港漢奸在港推動顏色革命 Over 1,000 US Consulate staffs, many with CIA launched Color Revolution to destabilize HK 香港警察「阿sir我撐你」 支持者逼爆添馬。\n\n多名藝人藍衣撐警 譚詠麟：家和萬事興。\n\n#香港 #撐警行動 https://t.co/alO0lhkMc6']


- @HKpoliticalnew, tweeting at 2019-06-22 22:16:00+08:00: 美國喺香港推動顏色革命 總指揮嚟自香港美國領事館 下令一定要制造流血事件 美國要見血讓其媒稱...


- @HKpoliticalnew; tweeting at 2019-06-13 23:38:00+08:00: '深扒內幕特工扮演記者傳媒其實是暗地裡擔任暴亂指揮工作有時故意扮採訪拍攝而擋在暴徒前面阻礙警察工作執勤反送中現場外國特工指揮暴亂顏色革命香港時政直擊'

- @KondratevFortu, tweeting at 2019-07-09 14:38:00+08:00 '有「佔中黑手」之稱的香港壹傳媒創辦人黎智英赴美勾連外國勢力干預香港事務。美國國務院發言人奧塔格斯（Morgan Ortagus）周一（8日）發新聞稿，表示美國國務卿蓬佩奧，當日於首都華盛頓與黎智英會晤，討論有關香港修訂《逃犯條例》的發展，以及香港在「一國兩制」框架下的自治地位等問題。 https://t.co/gHSxwgP1Xa'



## 4.2 SIFTING THROUGH TOP TROLL RETWEETS IN CHINESE

In [41]:
# This is the subset for retweets, in Chinese, by the top trolls
top_ch_trolls_rt = hk_ch_rt[hk_ch_rt['user_screen_name'].isin(trolls_ch)]
top_ch_trolls_rt.shape

(195, 21)

In [42]:
text = top_ch_trolls_rt["clean_tweet_text"].values
text_list = []
for t in text:
    text_list.append(' '.join(jieba.cut(t, HMM=False)))
    
vect = CountVectorizer(
    tokenizer=lambda x: x.split(), stop_words=jb_stopwords, min_df=10, ngram_range=(2, 4)
)
docs_ch = vect.fit_transform(text_list)
features_ch = vect.get_feature_names()

count_list6 = pd.DataFrame(docs_ch.toarray(),
                       columns=features_ch).sum(axis=0)
count_list6.sort_values(ascending = False).head(50)  

撐 警         57
香港 警察       47
社 會         37
行 動         34
香港 撐        33
我 哋         30
行 為         29
立法 會        27
警察 嘅        26
反 對         25
對 派         24
反 對 派       24
警 行 動       24
警 行         24
撐 警 行       24
撐 警 行 動     24
香港 撐 警      23
香港 撐 警 行    23
嚴 正         23
譴 責         23
執 法         22
哋 需要 你      20
警 隊         20
警 隊 你       20
警 隊 你 我     20
哋 需要        20
撐 警 隊 你     20
學 生         20
撐 警 隊       20
需要 你        20
我 同行        20
你 我 同行      20
隊 你 我 同行    20
隊 你 我       20
隊 你         20
你 我         20
我 哋 需要      20
我 哋 需要 你    20
逃犯 条例       16
穩 定         16
暴力 嘅        16
破 壞         16
香港 警察 嘅     16
正 執 法       14
社 會 治安      14
嚴 正 執 法     14
會 治安        14
連 續         14
嚴 正 執       14
正 執         14
dtype: int64

<img src="toptrolls_retweets.jpg">

In [43]:
# top_ch_trolls_rt[top_ch_trolls_rt["tweet_text"].str.contains('嚴正')].values
# top_ch_trolls_rt[top_ch_trolls_rt["tweet_text"].str.contains('撐警行動')].values
# top_ch_trolls_rt[top_ch_trolls_rt["tweet_text"].str.contains('社會治安')].values

The retweets continued the earlier trend of showing support for the HK police. This came frequently via the retweets of one of the top troll accounts @HKpoliticalnew. Uncomment the cell above for fuller details. A sample of the tweets:

- 'RT @HKpoliticalnew: 「我哋需要你」【1】\n\n連日嚟多個團體喺中環遮打花園等地舉辦撐警晚會、街站，精力更有大批人員到警總高呼「我哋需要你」，大叫「支持警察，嚴正執法，愛香港，撐警察」嘅口號。\n#香港 #撐警行動 https://t.co/1gp6JqhumY'

- 'RT @HKpoliticalnew: 暴徒們連續性嘅去街嚴重破壞了香港社會治安秩序，毆打執勤警員…\n7月1日暴徒們更系大肆使用暴力，肆意沖擊破壞立法會大樓，而反對派議員非但未阻止示威者武力升級，更叫警員「克制」，仲為暴徒提供人力及物資，煽風點火…\n呢種時候只能冀警方嚴正執法，…',

In Part 4, I'll use the Scattertext tool to produce interactive visualisations of the English troll tweets